# Deep learning on EEG Data

In [1]:
import sql_data_manager as sql
import torch
import torch.utils.data as torch_data
import fastai.tabular.all as fi
from fastai.metrics import mse, accuracy

KeyboardInterrupt: 

In [ ]:
database = sql.SqlDataSet("no_baseline_model.db")
#database = sql.ChunkedEdfDatabase(database, 160, seed=42, max_allowed_overlap=60, max_number_of_chunks=500)


In [ ]:
train_size = int(0.8 * len(database))
test_size = len(database) - train_size
train_database, test_database = torch_data.random_split(database, [train_size, test_size])

In [ ]:
class SimpleModel(torch.nn.Module):
    """Simple feed forward model. that predicts the labels for a second of eeg recordings.
        
    """
    def __init__(self) -> None:
        super(SimpleModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 2)
        self.lstm = torch.nn.LSTM(6*6*16, 512)
        self.mapping = torch.nn.Linear(512, 160)

    def forward(self, signals):
        # signals should be shaped as (w, h, channels, length)
        result = None
        hidden = (torch.randn(1, 1, 512), torch.randn(1, 1, 512))
        for slice in range(signals.shape[-1]):
            x = signals[:,:,:,slice] # x = (w, h, 1)
            x = self.conv1(x) # x = (w, h, 6)
            x = self.conv2(x) # x = (6, 6, 16)
            x = x.reshape((1,1,-1))
            result, hidden = self.lstm(x, hidden) # x = (512,)
        x = self.mapping(result) # x = (160,)
        return x

In [ ]:

class LstmModel(torch.nn.Module):
    def __init__(self) -> None:
        super(LstmModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, 2)
        self.lstm = torch.nn.LSTM(64, 128)
        self.hidden2tag = torch.nn.Linear(128, 1)

    def forward(self, signals: torch.Tensor):
        print("model", signals.shape)
        conv1_out = self.conv1(signals)
        print("conv1",conv1_out.shape)
        #flatten the signals to (1, signal_length, 64)
        view = conv1_out.view((1, signals.shape[0], 64))
        #print("view", view.shape)
        lstm_out, _ = self.lstm(view)
        #print("lstm", lstm_out.shape)
        tag_space = self.hidden2tag(lstm_out.view(signals.shape[0], -1))
        #print("tag_space", tag_space.shape)
        return tag_space

In [ ]:
dls = fi.DataLoaders(train_database, test_database)

In [ ]:
#model = SimpleModel()
model = LstmModel()
#model = torch.compile(model)

In [ ]:
learner = fi.Learner(dls, model, mse, metrics=[accuracy])

In [ ]:
learner.fit(5, 0.003)

epoch,train_loss,valid_loss,accuracy,time


model torch.Size([19680, 8, 8])


RuntimeError: Given groups=1, weight of size [6, 1, 2, 2], expected input[1, 19680, 8, 8] to have 1 channels, but got 19680 channels instead

In [ ]:
learner.predict(0)

AttributeError: 'Subset' object has no attribute 'new'

## Further work

Functioning model

Using a different dataset/train on different configurations of labels?
 - single imagination/pure movement
 - epilepsy classification

Better measurement of model (once the model works :) )
 - how it performs on unseen recordings/vs unseen humans